In [1]:
from pycocotools.coco import COCO
import requests
from PIL import Image
import glob
import os
import numpy as np

class Args:
    def __init__(self):
        self.root_path = '../data/coco/'
        self.class_list = ['airplane','suitcase','bird','cat','book','dog','elephant','horse','boat','truck']
        self.imgs_per_class = 300

args = Args()
test = True
if test:
    coco = COCO(args.root_path+'annotations/instances_val2017.json')
else:
    coco = COCO(args.root_path+'annotations/instances_train2017.json')

loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [2]:
## Return an array of image data and image labels with equal number images per class.
img_list = []
labels_list = []
k = 0
for i, class_name in enumerate(args.class_list):
    print('Now gathering images for class: ' + class_name)
    catIds = coco.getCatIds(catNms=class_name)
    imgIds = coco.getImgIds(catIds=catIds)
    imgs_info = coco.loadImgs(imgIds)
    n_imgs_in_class = 0

    for img_info in imgs_info:
        ## Download
        r = requests.get(img_info['coco_url'], stream=True)
        ## Image
        img = Image.open(r.raw)
        img = img.resize((32, 32), Image.BOX)
        img = np.array(img)
        img = np.expand_dims(img, 0)
        img_list.append(img)
        ## Label
        labels_list.append(catIds[0])
        ## Check number of images
        n_imgs_in_class += 1
        if n_imgs_in_class == args.imgs_per_class:
            break

Now gathering images for class: airplane
Now gathering images for class: suitcase
Now gathering images for class: bird
Now gathering images for class: cat
Now gathering images for class: book
Now gathering images for class: dog
Now gathering images for class: elephant
Now gathering images for class: horse
Now gathering images for class: boat
Now gathering images for class: truck


In [6]:
test = True

In [7]:
import pickle
if test:
    with open('../data/coco/image-data/test_img_data.pickle', 'wb') as F:
        pickle.dump(img_list, F)
    with open('../data/coco/image-data/test_labels.pickle', 'wb') as F:
        pickle.dump(labels_list, F)
else:
    with open('../data/coco/image-data/img_data.pickle', 'wb') as F:
        pickle.dump(img_list, F)
    with open('../data/coco/image-data/labels.pickle', 'wb') as F:
        pickle.dump(labels_list, F)

In [8]:
import copy
img_list2 = copy.copy(img_list)
for i,img in enumerate(img_list2):
    if img.ndim != 4:
        img_list2[i] = img_list2[i-1]

In [9]:
img_array = np.concatenate(img_list2, axis=0)
print(img_array.shape)
labels_array = np.hstack(labels_list)
print(labels_array.shape)

(1506, 32, 32, 3)
(1506,)


In [10]:
label_names = args.class_list
data = {'data': img_array, 'labels': labels_array, 'label_names': label_names}
if test:
    with open('../data/coco/image-data/test_data.pickle', 'wb') as F:
        pickle.dump(data, F)
else:
    with open('../data/coco/image-data/data.pickle', 'wb') as F:
        pickle.dump(data, F)